In [ ]:
!pip install pytesseract opencv-python
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
import cv2
import pytesseract
import re

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
G

In [ ]:
image = cv2.imread('/content/img1.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

**Applying Thresholding to improve image quality**

In [ ]:
extracted_text = pytesseract.image_to_string(gray_image)
print("Extracted Text:\n", extracted_text)

Extracted Text:
 BATCH NO.

USE BY

aa
= 470/- 21/08/24 20/08/25 F21B7
= 0.94 per 9g 14H4002401

(INCL. OF ALL TAXES)

eae

 



**Regex Code:**

In [ ]:
combined_brand_pattern = (
    r"(?i)(?:(?:Regd\.?\s*TM\s*of|Marketed\s*by|Manufactured\s*by|"
    r"A\s*company\s*of|Manufactured\s*&\s*Packed\s*by|Mfd\.?\s*by|@)[ \t]*|"
    r"(?:irademark Owner:[ \t]*|Trademark Owner:[ \t]*)|"
    r"\b(?:irademark Owner:[ \t]*|Trademark Owner:[ \t]*))"
    r"([A-Za-z&]+(?:\s+(?:Co\.?|PVT\.?|Pvt\.?|LTD\.?|Ltd\.?|LLC|INC\.?|GMBH|AG|CO\.))?)"  # Excluded digits by removing 0-9
    r"\b"
)
pack_size_pattern = r"(\d+\s?(N))"

# MRP pattern to capture both "MRP" mentioned values and values after '='
mrp_pattern = r"(?:MRP:?\s?(Rs\.?|₹|Rs\.:)?\s?(\d+\.\d{0,2})|=\s*(\d+))"
# Float pattern for standalone prices
float_pattern = r"\d+\.\d+"  # Matches float numbers

# The original brand_pattern variable was not defined, so I'm assuming it was intended to be combined_brand_pattern
brand_pattern = combined_brand_pattern

In [ ]:
from datetime import datetime
import re

def correct_ocr_date(date_str, ocr_correction):
    for incorrect, correct in ocr_correction.items():
        date_str = date_str.replace(incorrect, correct)
    return date_str

def extract_dates(text):
    # Regular expression to find dates in various formats (dd/mm/yyyy or mm/dd/yyyy)
    date_pattern = r'(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'
    matches = re.findall(date_pattern, text)
    return matches

def parse_dates(date_strings):
    parsed_dates = []
    ocr_correction = {
        '4': '1',  # Example: OCR may read 4 as 1
        '6': '0',  # Example: OCR may read 6 as 0
        '9': '3',  # Example: OCR may misinterpret 9 as 3
    }

    for date_str in date_strings:
        # Normalize the date string (replace / with - for consistency)
        date_str = date_str.replace('/', '-')

        # Split the date into components
        day, month, year = date_str.split('-')

        # Correct OCR misinterpretation only if day > 31 or month > 12
        if int(day) > 31 or int(month) > 12:
            corrected_date_str = correct_ocr_date(date_str, ocr_correction)
        else:
            corrected_date_str = date_str

        # Attempt to parse the date assuming dd-mm-yyyy format
        try:
            # If the year is only 2 digits, assume it's in the 21st century
            if len(year) == 2:
                corrected_date_str = corrected_date_str[:-2] + '20' + corrected_date_str[-2:]
            parsed_date = datetime.strptime(corrected_date_str, '%d-%m-%Y')
        except ValueError:
            # Attempt to parse the date assuming mm-dd-yyyy format if dd-mm-yyyy fails
            try:
                if len(year) == 2:
                    corrected_date_str = corrected_date_str[:-2] + '20' + corrected_date_str[-2:]
                parsed_date = datetime.strptime(corrected_date_str, '%m-%d-%Y')
            except ValueError:
                print(f"Error parsing date: {corrected_date_str}")
                continue
        parsed_dates.append(parsed_date)
    return parsed_dates

def get_latest_date(dates):
    if dates:
        return max(dates)  # Returns the latest (most recent) date
    return None

# Example usage:
extracted_dates = extract_dates(extracted_text)
parsed_dates = parse_dates(extracted_dates)
latest_expiry_date = get_latest_date(parsed_dates)

if latest_expiry_date:
    print("Latest Expiry Date:", latest_expiry_date.strftime('%d/%m/%Y'))
else:
    print("No valid expiry date found.")

Latest Expiry Date: 20/08/2025


In [ ]:
brand_match = re.search(brand_pattern, extracted_text, re.IGNORECASE)
if brand_match:
    brand_name = brand_match.group(1)
    print("Brand Name:", brand_name)
else:
    print("Brand Name not listed")

# Extract pack size
pack_size_match = re.search(pack_size_pattern, extracted_text, re.IGNORECASE)
if pack_size_match:
    pack_size = pack_size_match.group(0)
    print("Pack Size:", pack_size)
else:
    print("Pack Size: 1")

# Extract expiry date using the regex and apply the correction function
extracted_dates = extract_dates(extracted_text)
parsed_dates = parse_dates(extracted_dates)
latest_expiry_date = get_latest_date(parsed_dates)

if latest_expiry_date:
    print("Expiry Date:", latest_expiry_date.strftime('%d/%m/%Y'))
else:
    print("Expiry Date not listed.")


# Extract MRP
mrp_match = re.search(mrp_pattern, extracted_text, re.IGNORECASE)
if mrp_match and (mrp_match.group(2) or mrp_match.group(3)):  # Check for both decimal and whole number matches
    mrp_value = float(mrp_match.group(2)) if mrp_match.group(2) else int(mrp_match.group(3))  # Prioritize integer MRP
    formatted_mrp = f"MRP Rs. {mrp_value:.2f}" if isinstance(mrp_value, float) else f"MRP Rs. {mrp_value}"
    print("MRP:", formatted_mrp)
else:
    # If MRP not found, look for all float values including standalone values
    float_matches = re.findall(float_pattern, extracted_text)
    if float_matches:
        highest_mrp = max(float(float_value) for float_value in float_matches)
        print(f"MRP: Rs. {highest_mrp:.2f}")
    else:
        print("MRP not listed.")


Brand Name not listed
Pack Size: 1
Expiry Date: 20/08/2025
MRP: MRP Rs. 470
